In [ ]:
# import sys
# !{sys.executable} -m pip install pyyaml

In [1]:
import os
os.chdir(r'C:\Users\gizem\Desktop\master_thesis\source')

import pandas as pd
import yaml
from simulations.pomdp import POMDPSimulation
from utils.visualization import *
from utils.constants import *
from utils.helpers import *
from inference.sampling import *

In [2]:
df_all_traj = pd.read_csv(r'_data\inference_sampling\1583370639\df_all_traj.csv')
df_all_traj.drop('Unnamed: 0', axis=1, inplace=True)

df_traj = df_all_traj[(df_all_traj[TRAJ_ID]==1) & (df_all_traj[TIME]<=10)]
last = df_traj.iloc[-1]
last[TIME] = 10.0
df_traj_orig = df_traj.append(last)

c:\users\gizem\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
folder = create_folder_for_experiment(folder_name='../_data/inference_sampling/notebook/')

with open('configs/pomdp_sim.yaml', 'r') as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)

np.random.seed(cfg[SEED])
pomdp_sim = POMDPSimulation(cfg, save_folder=folder, import_data='1582930774')

In [10]:
np.random.seed(1)

phi_set = obs_model_set(len(cfg[STATES]), len(cfg[OBS_SPACE]))
# for obs_model in phi_set:
#     pomdp_sim.reset()
#     pomdp_sim.reset_obs_model(obs_model)

In [ ]:
pomdp_sim.reset()
pomdp_sim.reset_obs_model(phi_set[0])

In [11]:
df_traj = pomdp_sim.sample_parent_trajectory()

In [12]:
df_traj

,X,Y,time,o
0,1.0,1.0,0.000000,2
1,1.0,0.0,0.067504,1
2,1.0,1.0,0.078787,2
3,0.0,1.0,2.249753,1
4,1.0,1.0,2.413352,2
5,0.0,1.0,3.197805,1
6,0.0,0.0,3.218363,0
7,1.0,0.0,3.606313,1
8,1.0,1.0,4.779995,2
9,1.0,0.0,7.565133,1


In [13]:
prev = df_traj.iloc[0]
for i, row in df_traj.iterrows():
    if ((row == prev).all()) or (row[OBS] != prev[OBS]):
        pomdp_sim.update_belief_state_jump(int(row[OBS]), row[TIME])
    if i == df_traj.index[-1]:
        break
    else:
        t = row[TIME]
        t_next = df_traj[TIME].values[i + 1]
        pomdp_sim.append_event(t_next)
        pomdp_sim.update_cont_belief_state(t, t_next)

In [15]:
pomdp_sim.update_cont_Q()
pomdp_sim.df_Qz.to_csv('debug.csv')

In [16]:
for t_ in df_traj_.loc[abs(df_traj_['Z'].diff())>0, TIME]:
    pomdp_sim.append_event(t_)

In [17]:
node = 'Z'
df_Q = pomdp_sim.df_Qz.copy()

In [22]:
df_Q[T_DELTA] = np.append(to_decimal(pomdp_sim.time_grain),
                                         np.diff(df_Q.index)).astype(float)
df_Q.fillna(method='ffill', inplace=True)

In [26]:
L = 0
df_trans = df_traj_.loc[df_traj_[node].diff() != 0]
trans_tuples = list(
    zip(df_trans[node].astype(int), df_trans[node][1:].astype(int), df_trans[TIME], df_trans[TIME][1:]))

for i in trans_tuples:
    trans_tag = ''.join(map(str, i[0:2]))
    stay_tag = ''.join((str(i[0]), str(i[0])))
    times = i[2:4]
    q_trans = df_Q.truncate(after=to_decimal(times[1])).iloc[-1].loc[trans_tag]
    df_Q_ = df_Q.truncate(before=to_decimal(times[0]), after=to_decimal(times[1]))
    prob_stay = (df_Q_[stay_tag].multiply(df_Q_[T_DELTA], axis="index")).cumsum().loc[to_decimal(times[1])]
    prob_trans = df_Q.loc[to_decimal(times[1]), trans_tag] / abs(df_Q.loc[to_decimal(times[1]), stay_tag])
    L += np.log(q_trans) + np.log(prob_trans) + prob_stay

In [30]:
L_list = np.zeros((81))

In [31]:
L_list

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [33]:
df_all_traj[TRAJ_ID].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20], dtype=int64)